# 3. Anomaly Detection Model Training (Isolation Forest)

Notebook này train model Isolation Forest để phát hiện bất thường trong giá thuê.

## Mục tiêu:
- Phát hiện listings có giá quá cao/thấp so với đặc điểm
- Tìm các pattern bất thường (fraud, spam, giá giả mạo)
- Tạo anomaly score (càng âm = càng bất thường)

## Quy trình:
1. Load dữ liệu và XGBoost model
2. Tính predicted price cho mỗi property
3. Tạo features cho anomaly detection
4. Train Isolation Forest
5. Đánh giá và lưu model
6. Test với các trường hợp bất thường

## 1. Cài đặt thư viện

In [ ]:
!pip install scikit-learn pandas numpy matplotlib seaborn joblib

## 2. Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from google.colab import files

# Cấu hình
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')
sns.set_palette('husl')

## 3. Load dữ liệu và XGBoost model

Upload 2 files:
1. `training_data.csv` từ notebook 1
2. `price_model.pkl` từ notebook 2

In [ ]:
# Upload CSV
print("Upload training_data.csv:")
uploaded = files.upload()
csv_file = list(uploaded.keys())[0]

# Load data
df = pd.read_csv(csv_file)
print(f"\n✓ Đã load {len(df)} records")

# Upload price model
print("\nUpload price_model.pkl:")
uploaded = files.upload()
model_file = list(uploaded.keys())[0]

# Load price model
price_model_data = joblib.load(model_file)
price_model = price_model_data['model']
feature_columns = price_model_data['feature_columns']

print(f"\n✓ Đã load XGBoost model")
print(f"Test MAE: {price_model_data['metrics']['test_mae']:,.0f} VNĐ")
print(f"Test R²: {price_model_data['metrics']['test_r2']:.4f}")

## 4. Tạo features cho Anomaly Detection

In [ ]:
# Tính predicted price cho tất cả properties
X = df[feature_columns]
df['predicted_price'] = price_model.predict(X)

# Tính price difference (%)  
df['price_diff'] = df['price'] - df['predicted_price']
df['price_diff_percent'] = (df['price_diff'] / df['predicted_price']) * 100

# Price per square meter
df['price_per_sqm'] = df['price'] / df['area']
df['predicted_price_per_sqm'] = df['predicted_price'] / df['area']

# Z-score của price difference
df['price_diff_zscore'] = (df['price_diff'] - df['price_diff'].mean()) / df['price_diff'].std()

print("=== Price Prediction Statistics ===")
print(f"Mean difference: {df['price_diff'].mean():,.0f} VNĐ")
print(f"Std difference: {df['price_diff'].std():,.0f} VNĐ")
print(f"Mean difference %: {df['price_diff_percent'].mean():.2f}%")
print(f"\nProperties với giá lệch > 50%: {len(df[abs(df['price_diff_percent']) > 50])}")
print(f"Properties với giá lệch > 100%: {len(df[abs(df['price_diff_percent']) > 100])}")

In [ ]:
# Visualize price difference distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram of price difference %
axes[0, 0].hist(df['price_diff_percent'], bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Price Difference (%)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Distribution of Price Difference (%)')
axes[0, 0].axvline(x=0, color='r', linestyle='--', label='Perfect prediction')
axes[0, 0].legend()

# Boxplot
axes[0, 1].boxplot(df['price_diff_percent'])
axes[0, 1].set_ylabel('Price Difference (%)')
axes[0, 1].set_title('Boxplot of Price Difference')

# Actual vs Predicted
axes[1, 0].scatter(df['predicted_price'], df['price'], alpha=0.3)
axes[1, 0].plot([df['price'].min(), df['price'].max()],
                [df['price'].min(), df['price'].max()],
                'r--', lw=2)
axes[1, 0].set_xlabel('Predicted Price')
axes[1, 0].set_ylabel('Actual Price')
axes[1, 0].set_title('Actual vs Predicted Price')

# Residuals
axes[1, 1].scatter(df['predicted_price'], df['price_diff'], alpha=0.3)
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Predicted Price')
axes[1, 1].set_ylabel('Price Difference (VNĐ)')
axes[1, 1].set_title('Residual Plot')

plt.tight_layout()
plt.show()

## 5. Chuẩn bị features cho Isolation Forest

In [ ]:
# Chọn features cho anomaly detection
anomaly_features = [
    # Price-related
    'price',
    'predicted_price',
    'price_diff',
    'price_diff_percent',
    'price_per_sqm',
    'predicted_price_per_sqm',
    'price_diff_zscore',
    # Property features
    'area',
    'bedrooms',
    'bathrooms',
    # Location
    'province_encoded',
    'district_encoded',
    # Amenities
    'total_amenities',
    'amenity_score'
]

X_anomaly = df[anomaly_features]

# Chuẩn hóa features
scaler = StandardScaler()
X_anomaly_scaled = scaler.fit_transform(X_anomaly)

print(f"Features cho anomaly detection: {len(anomaly_features)}")
print(anomaly_features)
print(f"\nShape: {X_anomaly_scaled.shape}")

## 6. Train Isolation Forest

In [ ]:
# Train model với contamination = 0.1 (10% outliers)
print("=== Training Isolation Forest ===")

iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.1,  # Giả định 10% data là outliers
    max_samples='auto',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

iso_forest.fit(X_anomaly_scaled)

print("\n✓ Training hoàn tất!")

## 7. Đánh giá Model

In [ ]:
# Predict anomaly labels và scores
df['anomaly_label'] = iso_forest.predict(X_anomaly_scaled)  # 1 = normal, -1 = anomaly
df['anomaly_score'] = iso_forest.score_samples(X_anomaly_scaled)  # Càng âm = càng bất thường

# Thống kê
n_anomalies = len(df[df['anomaly_label'] == -1])
n_normal = len(df[df['anomaly_label'] == 1])

print("=== Anomaly Detection Results ===")
print(f"Normal: {n_normal} ({n_normal/len(df)*100:.1f}%)")
print(f"Anomalies: {n_anomalies} ({n_anomalies/len(df)*100:.1f}%)")
print(f"\nAnomaly score range: [{df['anomaly_score'].min():.4f}, {df['anomaly_score'].max():.4f}]")
print(f"Mean score (normal): {df[df['anomaly_label'] == 1]['anomaly_score'].mean():.4f}")
print(f"Mean score (anomaly): {df[df['anomaly_label'] == -1]['anomaly_score'].mean():.4f}")

In [ ]:
# Visualize anomaly scores
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram of anomaly scores
axes[0, 0].hist(df[df['anomaly_label'] == 1]['anomaly_score'], 
                bins=30, alpha=0.7, label='Normal', color='blue')
axes[0, 0].hist(df[df['anomaly_label'] == -1]['anomaly_score'], 
                bins=30, alpha=0.7, label='Anomaly', color='red')
axes[0, 0].set_xlabel('Anomaly Score')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Distribution of Anomaly Scores')
axes[0, 0].legend()

# Anomaly score vs price difference %
axes[0, 1].scatter(df[df['anomaly_label'] == 1]['anomaly_score'],
                   df[df['anomaly_label'] == 1]['price_diff_percent'],
                   alpha=0.5, label='Normal', s=20)
axes[0, 1].scatter(df[df['anomaly_label'] == -1]['anomaly_score'],
                   df[df['anomaly_label'] == -1]['price_diff_percent'],
                   alpha=0.7, label='Anomaly', s=50, color='red')
axes[0, 1].set_xlabel('Anomaly Score')
axes[0, 1].set_ylabel('Price Difference (%)')
axes[0, 1].set_title('Anomaly Score vs Price Difference')
axes[0, 1].legend()

# Price distribution
axes[1, 0].boxplot([df[df['anomaly_label'] == 1]['price'],
                     df[df['anomaly_label'] == -1]['price']],
                    labels=['Normal', 'Anomaly'])
axes[1, 0].set_ylabel('Price (VNĐ)')
axes[1, 0].set_title('Price Distribution by Label')

# Price diff % distribution
axes[1, 1].boxplot([df[df['anomaly_label'] == 1]['price_diff_percent'],
                     df[df['anomaly_label'] == -1]['price_diff_percent']],
                    labels=['Normal', 'Anomaly'])
axes[1, 1].set_ylabel('Price Difference (%)')
axes[1, 1].set_title('Price Difference by Label')

plt.tight_layout()
plt.show()

In [ ]:
# Phân tích các anomalies
anomalies = df[df['anomaly_label'] == -1].sort_values('anomaly_score')

print("=== Top 10 Most Anomalous Properties ===")
print("\n(Score càng âm = càng bất thường)\n")

for i, row in anomalies.head(10).iterrows():
    print(f"Property #{i}:")
    print(f"  Actual price: {row['price']:,.0f} VNĐ")
    print(f"  Predicted price: {row['predicted_price']:,.0f} VNĐ")
    print(f"  Difference: {row['price_diff_percent']:.1f}%")
    print(f"  Anomaly score: {row['anomaly_score']:.4f}")
    print(f"  Area: {row['area']} m², Bedrooms: {int(row['bedrooms'])}")
    print(f"  Price/m²: {row['price_per_sqm']:,.0f} VNĐ\n")

## 8. Xác định threshold cho moderation

In [ ]:
# Phân tích quantiles của anomaly score
quantiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
score_quantiles = df['anomaly_score'].quantile(quantiles)

print("=== Anomaly Score Quantiles ===")
for q, score in score_quantiles.items():
    count = len(df[df['anomaly_score'] <= score])
    print(f"{q*100:>5.1f}% quantile: {score:>8.4f} ({count:>4} properties)")

# Đề xuất threshold
print("\n=== Recommended Thresholds ===")
print("\nStrict (reject top 5% most anomalous):")
print(f"  threshold = {score_quantiles[0.05]:.4f}")
print(f"  → {len(df[df['anomaly_score'] <= score_quantiles[0.05]])} properties rejected")

print("\nModerate (reject top 10% most anomalous):")
print(f"  threshold = {score_quantiles[0.1]:.4f}")
print(f"  → {len(df[df['anomaly_score'] <= score_quantiles[0.1]])} properties rejected")

print("\nLenient (reject top 15% most anomalous):")
print(f"  threshold = {score_quantiles[0.15] if 0.15 in quantiles else df['anomaly_score'].quantile(0.15):.4f}")

## 9. Lưu Model

In [ ]:
# Lưu model và metadata
anomaly_model_data = {
    'model': iso_forest,
    'scaler': scaler,
    'anomaly_features': anomaly_features,
    'thresholds': {
        'strict': float(score_quantiles[0.05]),
        'moderate': float(score_quantiles[0.1]),
        'lenient': float(df['anomaly_score'].quantile(0.15))
    },
    'statistics': {
        'n_anomalies': int(n_anomalies),
        'contamination': 0.1,
        'score_range': [float(df['anomaly_score'].min()), 
                        float(df['anomaly_score'].max())]
    }
}

joblib.dump(anomaly_model_data, 'anomaly_model.pkl')
print("✓ Đã lưu model vào anomaly_model.pkl")

# Download về máy
files.download('anomaly_model.pkl')
print("\n✓ Đã tải model về máy!")
print("\nĐể sử dụng model:")
print("1. Upload file anomaly_model.pkl vào thư mục ml-moderation/models/")
print("2. Model sẽ tự động được load bởi ml_predictor.py")
print("\nLưu ý: Cần cả price_model.pkl và anomaly_model.pkl!")

## 10. Test với các trường hợp

In [ ]:
def test_property(area, bedrooms, bathrooms, amenities_count, price, 
                  province_encoded=1, district_encoded=5):
    """
    Test một property và trả về kết quả anomaly detection
    """
    # Tạo features cho price prediction
    total_amenities = amenities_count
    amenity_score = amenities_count / 10.0
    
    price_features = {
        'province_encoded': province_encoded,
        'district_encoded': district_encoded,
        'area': area,
        'bedrooms': bedrooms,
        'bathrooms': bathrooms,
        'floor': 2,
        'has_wifi': 1 if amenities_count >= 1 else 0,
        'has_parking': 1 if amenities_count >= 2 else 0,
        'has_air_conditioner': 1 if amenities_count >= 3 else 0,
        'has_water_heater': 1 if amenities_count >= 4 else 0,
        'has_kitchen': 1 if amenities_count >= 5 else 0,
        'has_fridge': 1 if amenities_count >= 6 else 0,
        'has_washing_machine': 1 if amenities_count >= 7 else 0,
        'has_tv': 1 if amenities_count >= 8 else 0,
        'has_bed': 1 if amenities_count >= 9 else 0,
        'has_wardrobe': 1 if amenities_count >= 10 else 0,
        'total_amenities': total_amenities,
        'amenity_score': amenity_score,
        'area_rooms': area * bedrooms,
        'area_amenities': area * amenity_score
    }
    
    # Predict price
    X_price = pd.DataFrame([price_features])
    predicted_price = price_model.predict(X_price)[0]
    
    # Tạo anomaly features
    price_diff = price - predicted_price
    price_diff_percent = (price_diff / predicted_price) * 100
    
    anomaly_data = {
        'price': price,
        'predicted_price': predicted_price,
        'price_diff': price_diff,
        'price_diff_percent': price_diff_percent,
        'price_per_sqm': price / area,
        'predicted_price_per_sqm': predicted_price / area,
        'price_diff_zscore': (price_diff - df['price_diff'].mean()) / df['price_diff'].std(),
        'area': area,
        'bedrooms': bedrooms,
        'bathrooms': bathrooms,
        'province_encoded': province_encoded,
        'district_encoded': district_encoded,
        'total_amenities': total_amenities,
        'amenity_score': amenity_score
    }
    
    # Predict anomaly
    X_anomaly = pd.DataFrame([anomaly_data])[anomaly_features]
    X_anomaly_scaled = scaler.transform(X_anomaly)
    
    anomaly_score = iso_forest.score_samples(X_anomaly_scaled)[0]
    anomaly_label = iso_forest.predict(X_anomaly_scaled)[0]
    
    # Kết quả
    return {
        'price': price,
        'predicted_price': predicted_price,
        'price_diff_percent': price_diff_percent,
        'anomaly_score': anomaly_score,
        'anomaly_label': 'Normal' if anomaly_label == 1 else 'Anomaly',
        'is_anomaly': anomaly_score < anomaly_model_data['thresholds']['moderate']
    }

# Test cases
test_cases = [
    {
        'name': 'Normal property',
        'area': 25,
        'bedrooms': 1,
        'bathrooms': 1,
        'amenities_count': 8,
        'price': 3000000
    },
    {
        'name': 'Overpriced (giá quá cao)',
        'area': 20,
        'bedrooms': 1,
        'bathrooms': 1,
        'amenities_count': 3,
        'price': 8000000  # Rất cao
    },
    {
        'name': 'Underpriced (giá quá thấp)',
        'area': 50,
        'bedrooms': 2,
        'bathrooms': 2,
        'amenities_count': 10,
        'price': 1500000  # Rất thấp
    },
    {
        'name': 'Luxury property',
        'area': 80,
        'bedrooms': 3,
        'bathrooms': 2,
        'amenities_count': 10,
        'price': 15000000
    }
]

print("=== Testing với các trường hợp ===")
print("\n" + "="*80)

for test_case in test_cases:
    result = test_property(
        area=test_case['area'],
        bedrooms=test_case['bedrooms'],
        bathrooms=test_case['bathrooms'],
        amenities_count=test_case['amenities_count'],
        price=test_case['price']
    )
    
    print(f"\n{test_case['name'].upper()}:")
    print(f"  Area: {test_case['area']} m², {test_case['bedrooms']} bedrooms")
    print(f"  Amenities: {test_case['amenities_count']}/10")
    print(f"  Actual price: {result['price']:,.0f} VNĐ")
    print(f"  Predicted price: {result['predicted_price']:,.0f} VNĐ")
    print(f"  Difference: {result['price_diff_percent']:+.1f}%")
    print(f"  Anomaly score: {result['anomaly_score']:.4f}")
    print(f"  Label: {result['anomaly_label']}")
    
    if result['is_anomaly']:
        print(f"  ⚠️  CẢNH BÁO: Giá bất thường - cần review!")
    else:
        print(f"  ✓ OK: Giá hợp lý")
    
    print("="*80)

## 11. Kết luận

Model Isolation Forest đã được train thành công!

**Hoàn thành:**
- ✓ Train Isolation Forest để phát hiện anomalies
- ✓ Xác định thresholds cho các mức độ nghiêm ngặt khác nhau
- ✓ Test với các trường hợp normal và anomalous
- ✓ Lưu model với metadata

**Cách hoạt động:**
- Anomaly score < threshold → Giá bất thường, cần review
- Score càng âm = càng bất thường
- Model kết hợp với XGBoost để so sánh giá thực tế vs dự đoán

**Bước tiếp theo:**
1. Upload cả `price_model.pkl` và `anomaly_model.pkl` vào `ml-moderation/models/`
2. Chạy Flask API server (notebook hoặc local)
3. Test integration với Node.js backend
4. Monitor và điều chỉnh thresholds dựa trên real data

**Thresholds được đề xuất:**
- Strict: {anomaly_model_data['thresholds']['strict']:.4f}
- Moderate: {anomaly_model_data['thresholds']['moderate']:.4f}
- Lenient: {anomaly_model_data['thresholds']['lenient']:.4f}